In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.metrics.pairwise import cosine_similarity
import re
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
def clean_function(resumeText):
    """ Cleaning / preprocessing text data"""
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

In [4]:
df = pd.read_csv("/home/dell/Downloads/archive/UpdatedResumeDataSet.csv")
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [5]:
df["Category"].value_counts()

Java Developer               84
Testing                      70
DevOps Engineer              55
Python Developer             48
Web Designing                45
HR                           44
Hadoop                       42
Blockchain                   40
ETL Developer                40
Operations Manager           40
Data Science                 40
Sales                        40
Mechanical Engineer          40
Arts                         36
Database                     33
Electrical Engineering       30
Health and fitness           30
PMO                          30
Business Analyst             28
DotNet Developer             28
Automation Testing           26
Network Security Engineer    25
SAP Developer                24
Civil Engineer               24
Advocate                     20
Name: Category, dtype: int64

In [6]:
Data_science = df.loc[df["Category"]== "Data Science"]
Data_science.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [7]:
Java_Developer = df.loc[df["Category"]== "Java Developer"]
Java_Developer.head()

,Category,Resume
319,Java Developer,Education Details \r\nAugust 2010 to May 2017 ...
320,Java Developer,Technical Skills (Trained / Project Acquired S...
321,Java Developer,"TECHNICAL SKILLS Skills: Java, SQL, PL/SQL, C,..."
322,Java Developer,TECHNICAL STRENGTHS Computer Language Java/J2E...
323,Java Developer,Education Details \r\nJanuary 2013 Master of E...


In [8]:
Testing = df.loc[(df["Category"]== "Testing")|(df["Category"]== "Automation Testing")]
Testing.head()


,Category,Resume
455,Automation Testing,* Excellent grasping power in learning new con...
456,Automation Testing,SOCIAL SKILLS: â¢ Ability to establish trust ...
457,Automation Testing,TECHNICAL SKILLS Automation Testing â¢ Seleni...
458,Automation Testing,"SKILLS Agile Methodology Scrum, Kanban, Extrem..."
459,Automation Testing,"Technical Skills Summary I have completed ""COR..."


In [9]:
Python_Developer = df.loc[df["Category"]== "Python Developer"]
Python_Developer.head()

,Category,Resume
551,Python Developer,Technical Skills / Responsibilities: â¢ Hands...
552,Python Developer,Education Details \r\nJune 2013 to June 2016 D...
553,Python Developer,TECHNICAL PROFICIENCIES Platform: Ubuntu/Fedor...
554,Python Developer,Technical Skills: Languages Python Python Fram...
555,Python Developer,Training attended: 1. Successfully completed E...


In [10]:
Web_Designing = df.loc[df["Category"]== "Web Designing"]
Web_Designing.head()

,Category,Resume
140,Web Designing,"Technical Skills Web Technologies: Angular JS,..."
141,Web Designing,Education Details \r\n B.C.A Bachelor Computer...
142,Web Designing,"Technical Skills Web Technologies: Angular JS,..."
143,Web Designing,Education Details \r\nJanuary 2016 B.Sc. Infor...
144,Web Designing,"IT SKILLS Languages: C (Basic), JAVA (Basic) W..."


In [11]:
Sales_1 = df.loc[df["Category"]== "Sales"]
Sales_1.head()

,Category,Resume
225,Sales,Education Details \r\n Bachelor's \r\n Bache...
226,Sales,SKILLS 1. MS-Office 2. Good Communication skil...
227,Sales,KEY SKILLS: â¢ Planning & Strategizing â¢ Pr...
228,Sales,IT Skills: MS Office. Photoshop. SQL Server.Ed...
229,Sales,Skill Sets: â¢ Multi-tasking â¢ Collaborativ...


In [12]:
Mechanical_Engineer = df.loc[df["Category"]== "Mechanical Engineer"]
Mechanical_Engineer.head()

,Category,Resume
185,Mechanical Engineer,Education Details \r\nMay 1999 to September 20...
186,Mechanical Engineer,SKILLS: â¢ Knowledge of software / computer: ...
187,Mechanical Engineer,Education Details \r\nJanuary 2018 Bachelor's ...
188,Mechanical Engineer,Education Details \r\nJune 2014 to June 2018 B...
189,Mechanical Engineer,* I'm hard working person. * I'm self confiden...


In [13]:
Civil_Engineer = df.loc[df["Category"]== "Civil Engineer"]
Civil_Engineer.head()

,Category,Resume
295,Civil Engineer,Education Details \r\n B.E in Civil Engineerin...
296,Civil Engineer,Education Details \r\nJanuary 2005 S.S.C Bara...
297,Civil Engineer,SKILLS â¢ 1.Autocad â¢ 2.Pro v â¢ 3.Catia â...
298,Civil Engineer,Computer Skills â¢ Holder of valid KSA licens...
299,Civil Engineer,COMPUTER KNOWLEDGE â¢ Drafting tools: AutoCAD...


In [40]:
first = pd.concat([Data_science,Java_Developer,Testing,Python_Developer,Web_Designing,Civil_Engineer,Mechanical_Engineer,Sales_1],ignore_index= True)
first

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
412,Sales,Education Details \r\n Bachelor's \r\n Bache...
413,Sales,SKILLS 1. MS-Office 2. Good Communication skil...
414,Sales,KEY SKILLS: â¢ Planning & Strategizing â¢ Pr...
415,Sales,IT Skills: MS Office. Photoshop. SQL Server.Ed...


# Description

In [15]:
df1 = pd.read_csv('jobs.csv',usecols=["Job Description","keyword"])
df1["keyword"].value_counts()

sales              43
project,manager    27
ux,designer        26
data,analyst       24
consultant         23
account,manager    22
product,manager    18
marketing          14
data,scientist     11
Name: keyword, dtype: int64

In [31]:
Sales = df1.loc[df1["keyword"]== "sales"]
Sales["keyword"] = "Sales"
Sales




,Job Description,keyword
165,We are looking for a talented and competitive ...,Sales
166,Our agreement with employees\n\nDocuSign is co...,Sales
167,Job Description*\nARIA Technologies is an indu...,Sales
168,Closet World* is a complete home organizing se...,Sales
169,American Solar has an IMMEDIATE OPENING for a ...,Sales
170,Mercedes-Benz of San Francisco is seeking Luxu...,Sales
171,Outside Sales Representative*\nIf you are seek...,Sales
172,Slakey Brothers is a wholesale distributor ded...,Sales
173,SSCS is looking for a highly motivated person ...,Sales
174,SALES COORDINATOR*\nSeeking driven and motivat...,Sales


In [30]:
data_scientist = df1.loc[df1["keyword"]== "data,scientist"]
data_scientist["keyword"] = "Data Science"
data_scientist


,Job Description,keyword
26,The Commercial Lines BI team at Woodruff Sawye...,Data Science
27,Our mission is to make the best mental health ...,Data Science
28,"About Figure\n\nAt Figure, we're on a mission ...",Data Science
29,Coursera is a leading online learning platform...,Data Science
30,Coursera is a leading online learning platform...,Data Science
31,"About Figure\n\nAt Figure, we're on a mission ...",Data Science
32,Company Description\n\nWish is a mobile e-comm...,Data Science
33,"At the intersection of technology, science, bu...",Data Science
34,Company Description\n\nWish is a mobile e-comm...,Data Science
35,Company Description\n\nWish is a mobile e-comm...,Data Science


In [19]:
ux_designer = df1.loc[df1["keyword"]== "ux,designer"]
ux_designer.head()


,Job Description,keyword
0,At Sonos we want to create the ultimate listen...,"ux,designer"
1,#TeamNextdoor\n\n\nNextdoor is the neighborhoo...,"ux,designer"
2,Description: *\nSPODIO is a data-driven sports...,"ux,designer"
3,SUMMARY OF POSITION\n\nThe UI/UX Designer is a...,"ux,designer"
4,"WHO WE ARE:\n\nFounded in 2012, BirdEye is hea...","ux,designer"


In [20]:
data_analyst = df1.loc[df1["keyword"]== "data,analyst"]
data_analyst.head()


,Job Description,keyword
37,Focused on labor and employment law since 1958...,"data,analyst"
38,Neustar is an information services and technol...,"data,analyst"
39,Clinical Data Programmer/Analyst supports clin...,"data,analyst"
40,Data Analyst (Client Success)*\nAre you lookin...,"data,analyst"
41,"@SYNNEX, we believe everyone has a story to te...","data,analyst"


In [32]:
second = pd.concat([Sales,ux_designer,data_analyst,data_scientist],ignore_index= True)
second

,Job Description,keyword
0,We are looking for a talented and competitive ...,Sales
1,Our agreement with employees\n\nDocuSign is co...,Sales
2,Job Description*\nARIA Technologies is an indu...,Sales
3,Closet World* is a complete home organizing se...,Sales
4,American Solar has an IMMEDIATE OPENING for a ...,Sales
...,...,...
99,Company Description\n\nWish is a mobile e-comm...,Data Science
100,"At the intersection of technology, science, bu...",Data Science
101,Company Description\n\nWish is a mobile e-comm...,Data Science
102,Company Description\n\nWish is a mobile e-comm...,Data Science


In [33]:
first.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [41]:
data = pd.DataFrame(zip(first["Resume"],first["Category"]), columns = ["Job Description","Jobs"])
data

,Job Description,Jobs
0,Skills * Programming Languages: Python (pandas...,Data Science
1,Education Details \r\nMay 2013 to May 2017 B.E...,Data Science
2,"Areas of Interest Deep Learning, Control Syste...",Data Science
3,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,Data Science
4,"Education Details \r\n MCA YMCAUST, Faridab...",Data Science
...,...,...
412,Education Details \r\n Bachelor's \r\n Bache...,Sales
413,SKILLS 1. MS-Office 2. Good Communication skil...,Sales
414,KEY SKILLS: â¢ Planning & Strategizing â¢ Pr...,Sales
415,IT Skills: MS Office. Photoshop. SQL Server.Ed...,Sales


In [42]:
data1 = pd.DataFrame(zip(second["Job Description"],second["keyword"]), columns = ["Job Description","Jobs"])
data1

,Job Description,Jobs
0,We are looking for a talented and competitive ...,Sales
1,Our agreement with employees\n\nDocuSign is co...,Sales
2,Job Description*\nARIA Technologies is an indu...,Sales
3,Closet World* is a complete home organizing se...,Sales
4,American Solar has an IMMEDIATE OPENING for a ...,Sales
...,...,...
99,Company Description\n\nWish is a mobile e-comm...,Data Science
100,"At the intersection of technology, science, bu...",Data Science
101,Company Description\n\nWish is a mobile e-comm...,Data Science
102,Company Description\n\nWish is a mobile e-comm...,Data Science


In [43]:
final_data = pd.concat([data,data1],ignore_index= True)

In [44]:
final_data

,Job Description,Jobs
0,Skills * Programming Languages: Python (pandas...,Data Science
1,Education Details \r\nMay 2013 to May 2017 B.E...,Data Science
2,"Areas of Interest Deep Learning, Control Syste...",Data Science
3,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,Data Science
4,"Education Details \r\n MCA YMCAUST, Faridab...",Data Science
...,...,...
516,Company Description\n\nWish is a mobile e-comm...,Data Science
517,"At the intersection of technology, science, bu...",Data Science
518,Company Description\n\nWish is a mobile e-comm...,Data Science
519,Company Description\n\nWish is a mobile e-comm...,Data Science


In [45]:
final_data["Jobs"].value_counts()

Java Developer         84
Sales                  83
Testing                70
Data Science           51
Python Developer       48
Web Designing          45
Mechanical Engineer    40
Automation Testing     26
ux,designer            26
Civil Engineer         24
data,analyst           24
Name: Jobs, dtype: int64

In [46]:
final_data.to_csv("Merged_data.csv")

In [47]:
df = pd.read_csv("Merged_data.csv")
df.drop("Unnamed: 0", inplace= True,axis = 1)
df

,Job Description,Jobs
0,Skills * Programming Languages: Python (pandas...,Data Science
1,Education Details \r\nMay 2013 to May 2017 B.E...,Data Science
2,"Areas of Interest Deep Learning, Control Syste...",Data Science
3,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,Data Science
4,"Education Details \r\n MCA YMCAUST, Faridab...",Data Science
...,...,...
516,Company Description\n\nWish is a mobile e-comm...,Data Science
517,"At the intersection of technology, science, bu...",Data Science
518,Company Description\n\nWish is a mobile e-comm...,Data Science
519,Company Description\n\nWish is a mobile e-comm...,Data Science


In [48]:
df["Jobs"].value_counts()

Java Developer         84
Sales                  83
Testing                70
Data Science           51
Python Developer       48
Web Designing          45
Mechanical Engineer    40
Automation Testing     26
ux,designer            26
Civil Engineer         24
data,analyst           24
Name: Jobs, dtype: int64

# Cosine similarity

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer 

from sklearn.metrics.pairwise import cosine_similarity 
tfidf_vectorizer = TfidfVectorizer() 

tfidf_matrix = tfidf_vectorizer.fit_transform(train_set) 

print(tfidf_matrix)

cosine = cosine_similarity(tfidf_matrix[length-1], tfidf_matrix) 

print(cosine)

NameError: name 'train_set' is not defined